In [1]:
import keras
keras.__version__

'2.4.3'

# 5.1 - Introduction to convnets

First, let's take a practical look at a very simple convnet example. We will use our convnet to classify MNIST digits,

convnet is a stack of Conv2D and MaxPooling2D layers. Importantly, a convnet takes as input tensors of shape (image_height, image_width, image_channels) (not including the batch dimension).


In [14]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation='relu',  input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, 3, activation='relu'))

In [15]:
# architecture of our convnet:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). 
The width and height dimensions tend to shrink as we go deeper in the network. 
The number of channels is controlled by the first argument passed to the Conv2D layers (e.g. 32 or 64).


The next step would be to feed our last output tensor (of shape (3, 3, 64)) into a densely-connected classifier network like those you are already familiar with: a stack of Dense layers. These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. So first, we will have to flatten our 3D outputs to 1D, and then add a few Dense layers on top:

In [18]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

We are going to do 10-way classification, so we use a final layer with 10 outputs and a softmax activation. Now here's what our network looks like:

In [19]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [24]:
#Now, let's train our convnet on the MNIST digit

In [20]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [21]:
train_images.shape

(60000, 28, 28, 1)

In [22]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 37s 39ms/step - loss: 0.7567 - accuracy: 0.7466
Epoch 2/5
938/938 [==============================] - 43s 46ms/step - loss: 0.0765 - accuracy: 0.9809
Epoch 3/5
938/938 [==============================] - 44s 46ms/step - loss: 0.0441 - accuracy: 0.9881
Epoch 4/5
938/938 [==============================] - 55s 59ms/step - loss: 0.0336 - accuracy: 0.9908
Epoch 5/5
938/938 [==============================] - 62s 66ms/step - loss: 0.0258 - accuracy: 0.9934


In [23]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 9ms/step - loss: 0.0297 - accuracy: 0.9921
